<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings # дополнительная библиотека для игнорирования предупреждений
import plotly.express as px

In [2]:
# Устанавливаем игнорирование разного рода предупреждений, чтобы сделать ноутбук более чистым 

categories_to_ignore = [
    DeprecationWarning,
    FutureWarning,
    UserWarning
]

for warn_ignore in categories_to_ignore:
    warnings.filterwarnings("ignore", category=warn_ignore)

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [10]:
# запрос на подсчет общего количества вакансий в базе
query_3_1 = f'''select 
                    count(*)
                from vacancies
            '''

In [28]:
# результат запроса
vacancy_qty = pd.read_sql_query(query_3_1, connection)

print(f'Всего в базе данных {vacancy_qty.loc[0][0]} вакансий')

Всего в базе данных 49197 вакансий


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [34]:
# текст запроса
query_emplrs_qty = '''select 
                count(id)
            from employers'''

In [37]:
# результат запроса
emplrs_qty = pd.read_sql_query(query_emplrs_qty, connection)

print(f'Всего в базе данных {emplrs_qty.loc[0][0]} работадателя')

Всего в базе данных 23501 работадателя


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [38]:
# текст запроса
query_areas_qty = '''select 
                count(id)
            from areas'''

In [39]:
# результат запроса
areas_qty = pd.read_sql_query(query_areas_qty, connection)

print(f'Всего в базе данных {areas_qty.loc[0][0]} региона')

Всего в базе данных 1362 региона


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [5]:
# текст запроса
query_industr_qty = '''select 
                count(id)
            from industries'''

In [6]:
# результат запроса
industr_qty = pd.read_sql_query(query_industr_qty, connection)

print(f'Всего в базе данных {industr_qty.loc[0][0]} сферы деятельности')

Всего в базе данных 294 сферы деятельности


***

Выводы: Всего в базе присутствует около 50_000 вакансий
В базе около 23_500 работадателей т.е в среднем по 2 вакансии на 1 работадателя
В базе 1362 региона, показатель достаточно большой значит дробление по регионам, города, поселкам и т.д.
В базе около 300 сфер деятельности что свидетельствует о достаточно разнородности данных

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [7]:
# текст запроса
query_areas_cnt = '''select 
                        ar.name area
                        ,count(vac.id) cnt
                    from vacancies vac
                        join areas ar on vac.area_id = ar.id
                    group by 1
                    order by 2 desc'''

In [8]:
# Вывел только первые 5 самых больших по числу вакансий региона, чтобы не захламлять ноутбук
areas_cnt = pd.read_sql_query(query_areas_cnt, connection)

areas_cnt.head()

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [9]:
# текст запроса
query_vacancy_with_salary = '''select 
                                    count(id)
                                from vacancies 
                                where 
                                    salary_from is not null 
                                    or salary_to is not null'''

In [10]:
# результат запроса
vacancy_with_salary = pd.read_sql_query(query_vacancy_with_salary, connection)

print(f'Количество вакансий которые имеют указание о зарплатном уровне {vacancy_with_salary.loc[0][0]}')

Количество вакансий которые имеют указание о зарплатном уровне 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [199]:
query_salary_level = '''select 
                            salary_from
                            ,salary_to
                        from vacancies 
                        where 
                            salary_from is not null 
                            or salary_to is not null'''
                                    

In [204]:
# Делаем запрос к базе, формируем 2 отдельные переменные для ответа на вопросы задания, выводим результат
salary_level = pd.read_sql_query(query_salary_level, connection)

salary_from_mean = round(salary_level.salary_from.mean(), 0)
salary_to_mean = round(salary_level.salary_to.mean(), 0)

print('Среднее значение для нижней границы равно {:.0f}, для верхней {:.0f}'.format(salary_from_mean, salary_to_mean))

Среднее значение для нижней границы равно 71065, для верхней 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [205]:
# текст запроса
query_schedule_employment = '''select 
                                    schedule
                                    ,employment
                                    ,count(id) vacancy_qty
                                from vacancies
                                group by 1, 2'''

In [207]:
# выводится датафрейм согласно условиям задания, решил отсортировать уже в питоне для разнообразия 
# (в тексте задания написано что результат отсортируйте по убыванию количества)
schedule_employment = pd.read_sql_query(query_schedule_employment, connection)

schedule_employment.sort_values(by='vacancy_qty', ascending=False).reset_index()

,index,schedule,employment,vacancy_qty
0,14,Полный день,Полная занятость,35367
1,11,Удаленная работа,Полная занятость,7802
2,9,Гибкий график,Полная занятость,1593
3,15,Удаленная работа,Частичная занятость,1312
4,12,Сменный график,Полная занятость,940
5,1,Полный день,Стажировка,569
6,16,Вахтовый метод,Полная занятость,367
7,7,Полный день,Частичная занятость,347
8,13,Гибкий график,Частичная занятость,312
9,8,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [208]:
# текст запроса 
query_vacancy_by_experience = '''select 
                                    experience
                                    ,count(id)
                                from vacancies
                                group by 1
                                order by 2 asc'''

In [209]:
# результат запроса 
vacancy_by_experience = pd.read_sql_query(query_vacancy_by_experience, connection)

vacancy_by_experience

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий следующие:
Больше всего вакансий в самых крупных городах в Москве и Питере ( за мкадом жизнь есть! но ее не так много:) )
Интересно что на третьем месте идет братский Минск
Лишь половина вакансий сопровождается данными по уровню дохода, тут возможны 3 варианта
Либо денег хотят предложить в диапазоне от "кот наплакал" до "и это еще до вычета налогов"
Либо бояться своих конкурентов, которые могут мониторить рынок и в частности своих крупных конкурентов,
чтобы можно было затянуть или ослабить гайки у себя, либо 3й вариант, для начала хотят прощупать кандидата
на синдром самозванца, так сказать поторговаться и нанять с минимальными затратами.
По зянятости ничего удивительного концепция "тело должно быть в офисе" с нами еще надолго, либо в данных 
большой процент вакансий по типу сталеваров, которые не могут работать без отрыва от места работы.
По опыту работы большая часть вопреки "вскукарекам" в ютубе таки берут или хотят людей с небольшим опытом,
а вот опытным кандидатам надо платить и тут уже вопрос бюджетов, не у всех они есть

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [20]:
# для выполнения задания использовал запрос с вертикальным объединением
# сперва до union находим 1е место потом объединяем со вторым запросом где уже меняем на 5 место
query_first_fifth_place_employeer = '''(select
                                            emp.name
                                            ,count(vac.id) vacancy_qty
                                        from vacancies vac
                                            join employers emp on emp.id = vac.employer_id
                                        group by 1
                                        order by 2 desc
                                        limit 1)
                                        union all
                                        (select
                                            emp.name
                                            ,count(vac.id) vacancy_qty
                                        from vacancies vac
                                            join employers emp on emp.id = vac.employer_id
                                        group by 1
                                        order by 2 desc
                                        limit 1 offset 4)'''

In [23]:
# дополнитльено создаем 2 переменные первого и пятого места и принтуем результат 
first_fifth_place_employeer = pd.read_sql_query(query_first_fifth_place_employeer, connection)

first_place = first_fifth_place_employeer.loc[0][0]
fifth_place = first_fifth_place_employeer.loc[1][0]

print(f'На первом месте находится {first_place}, а на пятом {fifth_place}')

На первом месте находится Яндекс, а на пятом Газпром нефть


,name,vacancy_qty
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# В условии задаем выборку только строчек где отсутствуют id вакансии, к таблице area справочнику 
# подтягиваем таблицы справочник по работадателям и таблицу с фактами по вакансиям

query_reqion_vacancy_employers = '''select 
                                        ar.name
                                        ,count(emp.id)
                                    from areas ar
                                        left join vacancies vac on vac.area_id = ar.id
                                        left join employers emp on emp.area = ar.id
                                    where vac.id is null
                                    group by 1
                                    order by 2 desc'''


In [26]:
# принтуем результат запроса и печатаем ответ на задание
reqion_vacancy_employers = pd.read_sql_query(query_reqion_vacancy_employers, connection)

print('Наибольшее количество работадателей в регионе где нет вакансий '
    f'{reqion_vacancy_employers.loc[0][0]}, '
    f'а количество работадателей равно {reqion_vacancy_employers.loc[0][1]}')

Наибольшее количество работадателей в регионе где нет вакансий Россия, а количество работадателей равно 410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [70]:
# запрос очень простой, джойним с 2 таблицами справочниками employear и areas 
# затем в селекте выбираем нужные колонки и применяем агрегацию с уникальным данным столбца id таблицы area
# группируем и сортируем по условию задачи
query_employer_region_qty = '''Select 
                                emp.name employer_name
                                ,count(distinct ar.id) reg_vac_qty
                            from vacancies vac
                                join employers emp on emp.id = vac.employer_id 
                                join areas ar on ar.id = vac.area_id
                            group by 1
                            order by 2 desc'''

In [75]:
# результат запроса вывожу первые 10 строк, чтобы не захламлять ноутбук
employer_region_qty = pd.read_sql_query(query_employer_region_qty, connection)

employer_region_qty.head(10)

,employer_name,reg_vac_qty
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [81]:
# запрос примитивный, к таблице справочнику лефт-джойним таблицу сфер-работадателей, где пусто там и есть наша нужная выборка
query_empl_wo_industry = '''select 
                                    count(id)
                                from employers emp
                                    left join employers_industries ind on emp.id = ind.employer_id
                                where industry_id is null'''

In [92]:
# выводим результат, тут я использовал патерн запроса сразу в переменную конечного ответа, чтобы не плодить строки
# метод read_sql_query возвращает датафрей, через ['count'] выводим серию и у серии выбираем самый первый вариант
# в нашем запросе он и единственный :) 
empl_wo_industry = pd.read_sql_query(query_empl_wo_industry, connection)['count'][0]

print(f'Количество работадателей которые не указали сферу деятельности {empl_wo_industry}')


Количество работадателей которые не указали сферу деятельности 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [6]:
# запрос также весьма простой, решил вытащить все данные, а уже питоном потом достать нужный ответ
query_forth_industries_companies = '''select 
                                        name
                                        ,count(industry_id)
                                    from employers emp
                                        left join employers_industries ind on emp.id = ind.employer_id
                                    group by 1
                                    having count(industry_id) = 4
                                    order by 1'''

In [13]:
# результат запроса
forth_industries_companies = pd.read_sql_query(query_forth_industries_companies, connection)

# получаем в переменную 3е место из датафрейма
third_company_in = forth_industries_companies.loc[2][0]

print('На третьем месте в алфавитном списке компаний у которых 4 сферы деятельности'
      f' находится компания {third_company_in}')

На третьем месте в алфавитном списке компаний у которых 4 сферы деятельности находится компания 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [15]:
# запрос простой джойним таблицу employear с таблицей пересечений работадателей и сфер,
# а также таблицей сфер и таблицей работадателей-сфер
query_employer_soft_developer = '''select 
                                        count(emp.id)
                                    from employers emp
                                        join employers_industries eind on emp.id = eind.employer_id
                                        join industries ind on ind.id = eind.industry_id
                                    where ind.name = 'Разработка программного обеспечения' '''

In [18]:
# просто выводим результат запроса
employer_soft_developer = pd.read_sql_query(query_employer_soft_developer, connection)

empl_softdev_qty = employer_soft_developer['count'][0]

print(f'В базе {empl_softdev_qty} работадателя у которах указана сфера деятельности разработка ПО')


В базе 3553 работадателя у которах указана сфера деятельности разработка ПО


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [143]:
# код для получения списка городов-милионников, 
# получаем готовый датафрейм по запросу из интернета, выбираем колонку с городом, переводим все в лист и помещаем
# в переменную mln_cities
# затем переводим все в str, чтобы подставить в наш запрос к нашей базе данных и помещаем в переменную mln_cities_str
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-' \
    '%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

mln_cities = list(pd.read_html(url)[1]['Город'])

mln_cities_str = ','.join(map(lambda x: "'" + x + "'", mln_cities))

In [147]:
# запрос предтавляет собой объединение 2х выборок, основной и добавляем через union тотальную цифру
# в запрос с помощью f-строк добавляем нашу переменную mln_cities_str
query_yandex_mln_cities = f'''   
                    (select 
                        ar.name
                        ,count(vac.id) cnt
                    from vacancies vac 
                        join areas ar on ar.id = vac.area_id
                        join employers emp on emp.id = vac.employer_id
                    where 
                        emp.name = 'Яндекс' and
                        ar.name in ({mln_cities_str})
                    group by 1
                    order by 2)

                    union 

                    (select 
                        'Total'
                        ,count(vac.id) cnt
                    from vacancies vac 
                        join areas ar on ar.id = vac.area_id
                        join employers emp on emp.id = vac.employer_id
                    where 
                        emp.name = 'Яндекс' and
                        ar.name in ({mln_cities_str})
                    )
                    order by 2
'''

In [149]:
# результат запроса
yandex_vacancy_by_mln_city = pd.read_sql_query(query_yandex_mln_cities, connection)
yandex_vacancy_by_mln_city

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

#### Выводы по анализу работодателей следущие:
Интересным представляется факт, что регионом с самым большим количеством работадателей у которых нет вакансий, 
является Россия, а это значит что большинство работадателей не указывают свой регион и просто ставят его как название страны.
что в целом наверное логично, т.к. уже непосредственно в описании вакансии указывается специфика региона, 
а тут мы как раз смотрим в их отсутствие.
В данных по некоторым работадателям присутствует детальное дробление по регионам вплоть до города.
так например Яндекс ищет работников в 181 регион
В базе присутствует достаточно большое количество работадателей которые не указали сферу своей деятельности.
выглядит это довольно странным, либо местному HR лень, либо HH денег за это просит, либо глобальный заговор
нет однозначности в понимании почему так происходит, гугл мне не помог в разрешении этого вопроса.
В данных присустствует около 10% работадателей занимающихся разработкой ПО
Выяснили, что компания яндекс (о неожиданность :) ) больше всего вакансий имеет в Москве, около 10% от общего
Что в целом не так уж и много учитывая размер населения города и статуса столицы, вероятно это связано
с тем что специалиста в Москву найти проще, поэтому вакансии закрываются активно, скажем девелопера найти
в Самару будет сложнее, так как страна у нас центрична, несмотря на все заявления о федерализме.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [150]:
# запрос весьма простой просто фильтруем таблицу с вакансиями по заданным условиям
query_data_specialists_qty = '''select count(id)
                                    from vacancies
                                where lower(name) like '%данн%' 
                                    or lower(name) like  '%data%' '''

In [156]:
# в результате 1771 вакансия это чуть больше 3% от общего количества 
data_specialists_qty = pd.read_sql_query(query_data_specialists_qty, connection)['count'][0]

print(f'В базе содержится {data_specialists_qty} вакансий имеющих отношение к данным')

В базе содержится 1771 вакансий имеющих отношение к данным


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [157]:
# фильтруем таблицу с вакансиями по заданному условию
query_junior_vacancies = '''select 
                                count(*)
                            from vacancies
                            where 
                                (lower(name) like '%data scientist%'
                                or lower(name) like '%data science%'
                                or lower(name) like '%исследователь данных%'
                                or upper(name) like '%ML%'
                                or lower(name) like '%machine learning%'
                                or lower(name) like '%машинн%обучен%')
                                and upper(name) not like '%HTML%'
                                and (lower(name) like '%junior%' or  
                                experience = 'Нет опыта' or employment = 'Стажировка')'''

In [163]:
# выводим результаты
junior_vacancies = pd.read_sql_query(query_junior_vacancies, connection)['count'][0]

print(f'В базе {junior_vacancies} вакансия удовлетворяющая условиям задачи')

В базе 51 вакансия удовлетворяющая условиям задачи


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [166]:
# фильтруем таблицу с вакансиями по заданному условию задачи
query_ds_vacancies_with_sql = '''select count(*)
                                        from vacancies
                                    where 
                                        (lower(name) like '%data scientist%'
                                        or lower(name) like '%data science%'
                                        or lower(name) like '%исследователь данных%'
                                        or upper(name) like '%ML%'
                                        or lower(name) like '%machine learning%'
                                        or lower(name) like '%машинн%обучен%')
                                        and upper(name) not like '%HTML%'
                                        and (lower(key_skills) like 
                                            '%sql%' or lower(key_skills) like '%postgres%')'''

In [169]:
# результат запроса
ds_vacancies_with_sql = pd.read_sql_query(query_ds_vacancies_with_sql, connection)['count'][0]
print(f'В базе содержится {ds_vacancies_with_sql} вакансий Data Scientist, где требуется навык SQL или Postgres')

В базе содержится 229 вакансий Data Scientist, где требуется навык SQL или Postgres


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [170]:
# фильтруем таблицу с вакансиями по заданному условию задачи
query_ds_python_skill_require = '''select count(*)
                                        from vacancies
                                    where 
                                        (lower(name) like '%data scientist%'
                                        or lower(name) like '%data science%'
                                        or lower(name) like '%исследователь данных%'
                                        or upper(name) like '%ML%'
                                        or lower(name) like '%machine learning%'
                                        or lower(name) like '%машинн%обучен%')
                                        and upper(name) not like '%HTML%'
                                        and (lower(key_skills) like '%python%')'''

In [173]:
# результат запроса
ds_python_skill_require = pd.read_sql_query(query_ds_python_skill_require, connection)['count'][0]
print(f'Требование по навыкам python имеется в {ds_python_skill_require} вакансиях на позицию DS')

Требование по навыкам python имеется в 357 вакансиях на позицию DS


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [176]:
# фильтруем таблицу с вакансиями по заданному условию задачи
# Интересным фактом, является "смердженные" данные в колонке клюбчевых навыков,
# а в качестве разделителя табуляция, никогда в своей практике работы с базами данных подобного не встречал
# интересно что по сути мы считает количество делиметров а не самих навыков, что в конечном результате, 
# безусловно дает такой же результат, т.к. мы сравниваем длинную 2х строк
query_ds_skills_qty = '''select 
                            round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2)
                        from vacancies
                        where
                            (name ilike '%data scientist%'
                            or name ilike '%data science%'
                            or name ilike '%исследователь данных%'
                            or name ilike '%machine learning%'
                            or name ilike '%машинн%обучен%')
                            or name like '%ML%' and name not ilike '%HTML%'
                            and key_skills is not null
                        '''

In [180]:
# результат запроса
ds_skills_qty = pd.read_sql_query(query_ds_skills_qty, connection)['round'][0]

print(f'Среднее количество навыков в вакансиях для DS -> {ds_skills_qty}')

Среднее количество навыков в вакансиях для DS -> 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [182]:
# Решил задачу с помощью конструкции case end, так привычнее решать подобные задачи
# в where предварительно убираем строчки где вообще не указана зарплата
query_ds_salary_by_experience = '''select 
                                    experience
                                    ,AVG(case 
                                        when salary_from is not null and salary_to is not null
                                            then (salary_from + salary_to) / 2
                                        when salary_from is not null 
                                            then salary_from
                                        when salary_to is not null 
                                            then salary_to end) avg_salary
                                from vacancies
                                where
                                    (name ilike '%data scientist%'
                                    or name ilike '%data science%'
                                    or name ilike '%исследователь данных%'
                                    or name ilike '%machine learning%'
                                    or name ilike '%машинн%обучен%'
                                    or (name like '%ML%' and name not ilike '%HTML%'))
                                    and (salary_from is not null or salary_to is not null)
                                group by 1'''

In [197]:
# Делаем коннект, достаем условие задачи
ds_salary_by_experience = pd.read_sql_query(query_ds_salary_by_experience, connection)
three_to_six_exp = ds_salary_by_experience[ds_salary_by_experience['experience'] == 'От 3 до 6 лет'].avg_salary.values[0]

print(f'Дата саентист с опытом 3-6 лет может рассчитывать за уровень дохода в среднем {round(three_to_six_exp, 0):.0f} тыс.руб')

Дата саентист с опытом 3-6 лет может рассчитывать за уровень дохода в среднем 243115 тыс.руб


***

#### Выводы по предметному анализу
В базе присутствует около 3х процентов вакансий, относящихся к анализу данных,
Из них джунов дата-саентистов "гулькин нос" 51 позиция (около 3х процентов от всех тех кто имеет отношения к данным)
и около 0,001% от всех вакансий, значит вкатится в айтишечку будут як как не просто :)
хорошая новость заключается в том, что все же можно устроится на работу с данными в том или ином виде.
Также кратно повышают шансы владения инструментами python и sql и др. 
Потенциальному кандидату нужно работать на увеличением своего скилового стэка, т.к. средний стэк в 
предпочтении работадателей это чуть более 6ти технологий.
Ну и на десерт, те кто таки вкатится в дата саенс уже через 3 года будут щедро вознаграждены сундуком с пиастрами,
тк. средняя зп таких специалистов около 250к в месяц, что в сравнении со средней по стране, неприлично много.

# Общий вывод по проекту

1. Перед нами полноценная база данных вакансий, которая содержит около 50_000 вакансий, 
2. Представлено около 23_500 работадалей, по более чем 300 сферам деятельности
3. Самое большое количество вакансий представлено в самых крупных городах миллионниках, география решает.
4. Не у всех работадателей есть открытые вакансии
5. Для потенциального кандидата, будет интересным факт, что новички на рынке требуются, но для этого нужно иметь стек знаний, который можно развивать, в анализе данных это sql и python, приветствуется чтобы кандидат имел более 6 навыков.
6. Хотя и имея стек очень хорошо, но позиции именно специализированные data science все же в массе свой треюуется опыт работы, вероятно стоит начать с более простых вакансий например аналитик данных, заработать опыт и уже потом двигаться дальше.

In [51]:
# Интересно дополнительно узнать насколько для работадателя является обязательным наличие высшего образования
# на позиции связанные с данными

query_data_specialist_vacancy = '''select *
                                    from vacancies
                                    where lower(name) like '%данн%' 
                                    or lower(name) like  '%data%' '''

In [57]:
# проводим необходимые исследования в данных
data_specialist_df = pd.read_sql_query(query_data_specialist_vacancy, connection)

data_specialist_df['key_skills'].fillna('', inplace=True)

# находим вакансии где указано высшее образование
higher_education = data_specialist_df[data_specialist_df
                                      .key_skills.str.contains('высшее образование', case=False)].shape[0]

portion_of_higher_education = higher_education / data_specialist_df.shape[0]

print('Количество вакансий связанные с работой с данными в которых работадатели \n указали '
    f'наличие высшего образования {portion_of_higher_education * 100:.2f}% от общего количества таких вакансий')

Количество вакансий связанные с работой с данными в которых работадатели 
 указали наличие высшего образования 0.28% от общего количества таких вакансий


In [6]:
# Выясняем, а по каким сферам в основном представлены вакансии работадателей
# предварительно создаем вложенный запрос с работадателями и сферами в которых они работают.
# поскольку один работадатель может иметь несколько сфер, пришлось агрегировать сферы через string_agg
# очень полезная функция на практике не раз выручала, странно но в курсе она представлена не была
# ну и затем получившуюся таблицу сфер и работадателей присоединяем к таблице с вакансиями
query_industry_vacancy_qty = '''select 
                                    coalesce(emp_inds.industry_group, 'Сфера деятельности не указана') as industry_group
                                    ,count(distinct vac.id) vacancy_qty
                                from vacancies vac
                                    join 
                                        (select 
                                            emp.id
                                            ,string_agg(ind.name, ';') industry_group
                                        from employers emp 
                                            left join employers_industries emp_ind on emp_ind.employer_id = emp.id
                                            left join industries ind on ind.id = emp_ind.industry_id
                                        group by 1) emp_inds
                                    on emp_inds.id = vac.employer_id
                                group by 1
                                order by 2 desc'''

In [34]:
# датасет достаточно массивный, решил сделать фокус на самых крупных
industry_vacancy_qty = pd.read_sql_query(query_industry_vacancy_qty, connection )
vac_ttl = industry_vacancy_qty.vacancy_qty.sum()
vac_sixty_five_perc = vac_ttl * 0.65 # не совсем правило Парето но очень близко к нему :)
# выбираем только самые крупные сферы из нашего датасета
industry_vacancy_qty = industry_vacancy_qty[industry_vacancy_qty.vacancy_qty.cumsum() <= vac_sixty_five_perc]

#создаем колонку с укороченным названием сферы
industry_vacancy_qty['industry_group_short'] = industry_vacancy_qty.industry_group.str[0:30] 
# вновь группируем по коротким названиям сфер в новый датасет
industry_vac_short = industry_vacancy_qty.groupby('industry_group_short', as_index=False)['vacancy_qty'].sum() 
# сортируем новый датасет
industry_vac_short.sort_values(by='vacancy_qty', ascending=False, inplace=True)

fig = px.bar(industry_vac_short, 
             x='industry_group_short',
             y='vacancy_qty',
             height=700
             ,title='Количество вакансий по сферам')

fig.update_layout(
    xaxis=dict(
        tickfont=dict(
            size=6,  # Установка размера подписей делений
        ),
        ticklen=1,        # Установка длины делений
    ),
    yaxis=dict(
        tickfont=dict(
            size=6,  # Установка размера подписей делений
        ),
        ticklen=1,  # Установка длины делений
    )
)

connection.close()

Как видно из дополнительного анализа, наличие высшего образование почти не играет никакой роли в сфере работы с данными.
Большая часть вакансий в нашей базе так или иначе относится к сфере IT, тут и разработчики и аналитики и банки и много другое. Также видно, что есть огромная часть неопределившихся работадателей (чем они занимаются они не указывают), для протенциального кандидата, ситуация выглядит следующим образом, видно, что технологическая эра приближается с высокой скоростью, работадателям требуется все большее и большее количество работников, которые будут способны выполнять все более сложные технические задачи, поэтому на первый план выходят знания кандидатов, ставку работадатели делают именно на качество знаний и количество технологий, больше чем на документальные подтверждения, т.к. это не всегда одно и тоже.
